In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

# Set random seed for reproducibility
np.random.seed(42)

# Load dataset
# Note: You'll need to replace this path with the location of your earthquake dataset
df = pd.read_csv(r"C:\Users\jidub\OneDrive\Documents\Timeseries\project\Eartquakes-1990-2023.csv\Eartquakes-1990-2023.csv")

# Preprocess timestamp
df['time'] = pd.to_datetime(df['time'], unit='ms')
df = df.sort_values('time')  # Ensure data is chronologically ordered
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour

# Handle missing values
df.fillna(method='ffill', inplace=True)

# === Feature Engineering ===
# 1. Add cyclical time features
df['month_sin'] = np.sin(df['month'] * (2 * np.pi / 12))
df['month_cos'] = np.cos(df['month'] * (2 * np.pi / 12))
df['hour_sin'] = np.sin(df['hour'] * (2 * np.pi / 24))
df['hour_cos'] = np.cos(df['hour'] * (2 * np.pi / 24))
df['day_sin'] = np.sin(df['day'] * (2 * np.pi / 31))
df['day_cos'] = np.cos(df['day'] * (2 * np.pi / 31))

# 2. Rolling statistics
for window in [7, 30, 90]:
    df[f'rolling_mag_mean_{window}d'] = df['magnitudo'].rolling(window=window).mean()
    df[f'rolling_depth_mean_{window}d'] = df['depth'].rolling(window=window).mean()
    df[f'rolling_quake_count_{window}d'] = df['magnitudo'].rolling(window=window).count()

# 3. Handle outliers
def cap_outliers(series, lower_quantile=0.01, upper_quantile=0.99):
    lower_bound = series.quantile(lower_quantile)
    upper_bound = series.quantile(upper_quantile)
    return series.clip(lower=lower_bound, upper=upper_bound)

df['magnitudo'] = cap_outliers(df['magnitudo'])
df['depth'] = cap_outliers(df['depth'])

# 4. Create interaction features
df['lat_lon_interaction'] = df['latitude'] * df['longitude'] / 10000  # Scaled down

# 5. Create magnitude categories
df['mag_low'] = (df['magnitudo'] < 2.0).astype(int)
df['mag_medium'] = ((df['magnitudo'] >= 2.0) & (df['magnitudo'] < 4.0)).astype(int)
df['mag_high'] = (df['magnitudo'] >= 4.0).astype(int)

# 6. Add geographic region clustering
def assign_region(lat, lon):
    # Simplified region assignment based on latitude/longitude
    if lat > 30 and lon > 120:  # Pacific Ring of Fire - Asia side
        return 1
    elif lat > 30 and lon < -100:  # Pacific Ring of Fire - Americas side
        return 2
    elif -30 < lat < 30:  # Equatorial regions
        return 3
    else:  # Other regions
        return 4

df['region'] = df.apply(lambda x: assign_region(x['latitude'], x['longitude']), axis=1)
df = pd.get_dummies(df, columns=['region'], prefix='region')

# Fill any remaining NaN values
df.fillna(method='bfill', inplace=True)

# Define features
features = [
    'latitude', 'longitude', 'depth',  
    'month_sin', 'month_cos', 'day_sin', 'day_cos', 'hour_sin', 'hour_cos',
    'rolling_mag_mean_30d', 'rolling_depth_mean_30d', 'rolling_quake_count_30d',
    'rolling_mag_mean_7d', 'rolling_mag_mean_90d',
    'lat_lon_interaction', 'mag_low', 'mag_medium', 'mag_high',
    'region_1', 'region_2', 'region_3', 'region_4'
]

targets = ['latitude', 'longitude', 'magnitudo']

# Extract features and targets
X = df[features].values
y = df[targets].values

# Use time-based split for temporal data
test_size = int(0.1 * len(X))
val_size = int(0.1 * len(X))

X_train = X[:-test_size-val_size]
X_val = X[-test_size-val_size:-test_size]
X_test = X[-test_size:]

y_train = y[:-test_size-val_size]
y_val = y[-test_size-val_size:-test_size]
y_test = y[-test_size:]

# Scale the features
feature_scaler = RobustScaler()
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Random Forest doesn't require target scaling, but we'll scale for evaluation consistency
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train)
y_val_scaled = target_scaler.transform(y_val)
y_test_scaled = target_scaler.transform(y_test)

# Train a separate Random Forest model for each target
models = []
target_names = ['Latitude', 'Longitude', 'Magnitude']

for i in range(3):
    print(f"\nTraining Random Forest for {target_names[i]}...")
    
    # Create and train the model
    rf_model = RandomForestRegressor(
        n_estimators=100,        # Number of trees
        max_depth=15,            # Maximum depth of trees
        min_samples_split=5,     # Minimum samples required to split
        min_samples_leaf=2,      # Minimum samples required at leaf node
        max_features='sqrt',     # Max features to consider for best split
        n_jobs=-1,               # Use all available processors
        random_state=42          # For reproducibility
    )
    
    # Fit the model to the training data
    rf_model.fit(X_train_scaled, y_train_scaled[:, i])
    models.append(rf_model)
    
    # Make predictions on validation set to check performance
    val_pred = rf_model.predict(X_val_scaled)
    val_mse = mean_squared_error(y_val_scaled[:, i], val_pred)
    val_rmse = np.sqrt(val_mse)
    val_mae = mean_absolute_error(y_val_scaled[:, i], val_pred)
    val_r2 = r2_score(y_val_scaled[:, i], val_pred)
    
    print(f"Validation RMSE: {val_rmse:.4f}")
    print(f"Validation MAE: {val_mae:.4f}")
    print(f"Validation R²: {val_r2:.4f}")

# Evaluate models on test set
print("\n=== Random Forest Model Performance ===")

for i, (model, target_name) in enumerate(zip(models, target_names)):
    # Make predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    
    # Convert predictions to original scale for a single target
    y_train_true_single = y_train_scaled[:, i]
    y_test_true_single = y_test_scaled[:, i]
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train_true_single, y_train_pred)
    test_mse = mean_squared_error(y_test_true_single, y_test_pred)
    train_rmse = np.sqrt(train_mse)
    test_rmse = np.sqrt(test_mse)
    train_mae = mean_absolute_error(y_train_true_single, y_train_pred)
    test_mae = mean_absolute_error(y_test_true_single, y_test_pred)
    train_r2 = r2_score(y_train_true_single, y_train_pred)
    test_r2 = r2_score(y_test_true_single, y_test_pred)
    
    # Overfitting ratio
    mse_overfitting_ratio = test_mse / train_mse if train_mse > 0 else float('inf')
    
    print(f"\n{target_name} Performance:")
    print(f"  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}")
    print(f"  Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
    print(f"  Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}")
    print(f"  MSE Overfitting Ratio (Test/Train): {mse_overfitting_ratio:.2f}")

# Visualize feature importances
plt.figure(figsize=(14, 10))

for i, (model, target_name) in enumerate(zip(models, target_names)):
    plt.subplot(3, 1, i+1)
    
    # Get feature importances
    importances = model.feature_importances_
    indices = np.argsort(importances)
    
    plt.barh(range(len(indices)), importances[indices], color='skyblue')
    plt.yticks(range(len(indices)), [features[i] for i in indices])
    plt.title(f'Feature Importance for {target_name}')
    plt.xlabel('Relative Importance')
    plt.tight_layout()

plt.savefig('random_forest_feature_importance.png')
plt.close()

# Visualize predictions vs actual values
plt.figure(figsize=(15, 15))

# Use a subset of test data for clearer visualization
test_indices = np.random.choice(range(len(y_test_scaled)), min(100, len(y_test_scaled)), replace=False)

for i, (model, target_name) in enumerate(zip(models, target_names)):
    plt.subplot(3, 1, i+1)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    
    # Plot
    plt.plot(y_test_scaled[test_indices, i], label='Actual', marker='o', linestyle='', alpha=0.7)
    plt.plot(y_pred[test_indices], label='Predicted', marker='x', linestyle='', alpha=0.7)
    plt.title(f'{target_name} - Actual vs Predicted')
    plt.xlabel('Sample Index')
    plt.ylabel(target_name)
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.savefig('random_forest_predictions.png')
plt.close()

# Visualize error distribution
plt.figure(figsize=(15, 5))

for i, (model, target_name) in enumerate(zip(models, target_names)):
    plt.subplot(1, 3, i+1)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    
    # Calculate errors
    errors = y_test_scaled[:, i] - y_pred
    
    # Plot error distribution
    plt.hist(errors, bins=30, color=['skyblue', 'lightgreen', 'salmon'][i], edgecolor='black')
    plt.title(f'{target_name} Error Distribution')
    plt.xlabel('Error')
    plt.axvline(x=0, color='red', linestyle='--')

plt.tight_layout()
plt.savefig('random_forest_error_distribution.png')
plt.close()

# Function to make predictions with all three models
def predict_with_random_forest(models, input_data, feature_scaler, target_scaler):
    """
    Make predictions using all three random forest models
    
    Parameters:
    models: List of trained Random Forest models [lat_model, lon_model, mag_model]
    input_data: Numpy array of shape (n_samples, n_features)
    feature_scaler: Fitted scaler for input features
    target_scaler: Fitted scaler for target variables
    
    Returns:
    prediction: Dictionary with predicted latitude, longitude, and magnitude
    """
    # Scale the input data
    scaled_input = feature_scaler.transform(input_data)
    
    # Get predictions from each model
    lat_pred = models[0].predict(scaled_input)
    lon_pred = models[1].predict(scaled_input)
    mag_pred = models[2].predict(scaled_input)
    
    # Combine predictions
    combined_pred = np.column_stack((lat_pred, lon_pred, mag_pred))
    
    # If you need to convert back to original scale:
    # original_pred = target_scaler.inverse_transform(combined_pred)
    
    return {
        'latitude': lat_pred,
        'longitude': lon_pred,
        'magnitude': mag_pred
    }

print("\nRandom Forest models trained successfully. Use the predict_with_random_forest() function to make new predictions.")

# Compare Random Forest with Deep Learning performance
print("\n=== Random Forest vs. Deep Learning Performance Comparison ===")
print("Note: The deep learning model results need to be manually compared from the previous output.")
print("General advantages of Random Forest for this dataset:")
print("1. Faster training time")
print("2. Less prone to overfitting with proper tuning")
print("3. Better interpretability through feature importance")
print("4. Can handle non-linear relationships without extensive feature engineering")
print("5. No need for sequence processing or time window preparation")

In [ ]:
import numpy as np
import pandas as pd
import pywt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load dataset
df = pd.read_csv(r"C:\Users\jidub\OneDrive\Documents\Timeseries\project\Eartquakes-1990-2023.csv\Eartquakes-1990-2023.csv")
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.set_index('date', inplace=True)

data = df['magnitudo'].resample('M').mean().dropna()

# Normalize data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.values.reshape(-1, 1)).flatten()

# Apply Discrete Wavelet Transform (DWT)
wavename = 'db4'  # Daubechies wavelet
coeffs = pywt.wavedec(data_scaled, wavename, level=3)
cA3, cD3, cD2, cD1 = coeffs  # Approximation and details

# Reconstruct smoothed signal
smoothed_data = pywt.waverec([cA3] + [None] * 3, wavename)[:len(data_scaled)]

# Prepare sequences for LSTM
sequence_length = 30
X, y = [], []
for i in range(len(smoothed_data) - sequence_length):
    X.append(smoothed_data[i:i+sequence_length])
    y.append(smoothed_data[i+sequence_length])
X, y = np.array(X), np.array(y)

# Reshape input for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split into train and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build LSTM model
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(sequence_length, 1)),
    Dropout(0.3),
    LSTM(100, return_sequences=False),
    Dropout(0.3),
    Dense(1)
])

# Compile and train model
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Forecasting
lstm_forecast = model.predict(X_test[-30:])
lstm_forecast = scaler.inverse_transform(lstm_forecast.reshape(-1, 1))

# Evaluate model
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
y_test_predicted = scaler.inverse_transform(model.predict(X_test))

mae = mean_absolute_error(y_test_actual, y_test_predicted)
rmse = np.sqrt(mean_squared_error(y_test_actual, y_test_predicted))
mape = np.mean(np.abs((y_test_actual - y_test_predicted) / y_test_actual)) * 100
accuracy = 100 - mape  # Accuracy as (100 - MAPE)

# Print results
print(f"Hybrid LSTM-Wavelet Model Accuracy:")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Forecast Accuracy: {accuracy:.2f}%")


In [ ]:
Combined Code

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
import pywt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load dataset
# Note: You'll need to replace this path with the location of your earthquake dataset
df = pd.read_csv(r"C:\Users\jidub\OneDrive\Documents\Timeseries\project\Eartquakes-1990-2023.csv\Eartquakes-1990-2023.csv")

# Preprocess timestamp
df['time'] = pd.to_datetime(df['time'], unit='ms')
df = df.sort_values('time')  # Ensure data is chronologically ordered
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour

# Handle missing values
df.fillna(method='ffill', inplace=True)

# === Feature Engineering ===
# 1. Add cyclical time features
df['month_sin'] = np.sin(df['month'] * (2 * np.pi / 12))
df['month_cos'] = np.cos(df['month'] * (2 * np.pi / 12))
df['hour_sin'] = np.sin(df['hour'] * (2 * np.pi / 24))
df['hour_cos'] = np.cos(df['hour'] * (2 * np.pi / 24))
df['day_sin'] = np.sin(df['day'] * (2 * np.pi / 31))
df['day_cos'] = np.cos(df['day'] * (2 * np.pi / 31))

# 2. Rolling statistics
for window in [7, 30, 90]:
    df[f'rolling_mag_mean_{window}d'] = df['magnitudo'].rolling(window=window).mean()
    df[f'rolling_depth_mean_{window}d'] = df['depth'].rolling(window=window).mean()
    df[f'rolling_quake_count_{window}d'] = df['magnitudo'].rolling(window=window).count()

# 3. Handle outliers
def cap_outliers(series, lower_quantile=0.01, upper_quantile=0.99):
    lower_bound = series.quantile(lower_quantile)
    upper_bound = series.quantile(upper_quantile)
    return series.clip(lower=lower_bound, upper=upper_bound)

df['magnitudo'] = cap_outliers(df['magnitudo'])
df['depth'] = cap_outliers(df['depth'])

# 4. Create interaction features
df['lat_lon_interaction'] = df['latitude'] * df['longitude'] / 10000  # Scaled down

# 5. Create magnitude categories
df['mag_low'] = (df['magnitudo'] < 2.0).astype(int)
df['mag_medium'] = ((df['magnitudo'] >= 2.0) & (df['magnitudo'] < 4.0)).astype(int)
df['mag_high'] = (df['magnitudo'] >= 4.0).astype(int)

# 6. Add geographic region clustering
def assign_region(lat, lon):
    # Simplified region assignment based on latitude/longitude
    if lat > 30 and lon > 120:  # Pacific Ring of Fire - Asia side
        return 1
    elif lat > 30 and lon < -100:  # Pacific Ring of Fire - Americas side
        return 2
    elif -30 < lat < 30:  # Equatorial regions
        return 3
    else:  # Other regions
        return 4

df['region'] = df.apply(lambda x: assign_region(x['latitude'], x['longitude']), axis=1)
df = pd.get_dummies(df, columns=['region'], prefix='region')

# Fill any remaining NaN values
df.fillna(method='bfill', inplace=True)

# Define features
features = [
    'latitude', 'longitude', 'depth',  
    'month_sin', 'month_cos', 'day_sin', 'day_cos', 'hour_sin', 'hour_cos',
    'rolling_mag_mean_30d', 'rolling_depth_mean_30d', 'rolling_quake_count_30d',
    'rolling_mag_mean_7d', 'rolling_mag_mean_90d',
    'lat_lon_interaction', 'mag_low', 'mag_medium', 'mag_high',
    'region_1', 'region_2', 'region_3', 'region_4'
]

targets = ['latitude', 'longitude', 'magnitudo']

# Extract features and targets
X = df[features].values
y = df[targets].values

# Use time-based split for temporal data
test_size = int(0.1 * len(X))
val_size = int(0.1 * len(X))

X_train = X[:-test_size-val_size]
X_val = X[-test_size-val_size:-test_size]
X_test = X[-test_size:]

y_train = y[:-test_size-val_size]
y_val = y[-test_size-val_size:-test_size]
y_test = y[-test_size:]

# Scale the features
feature_scaler = RobustScaler()
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Random Forest doesn't require target scaling, but we'll scale for evaluation consistency
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train)
y_val_scaled = target_scaler.transform(y_val)
y_test_scaled = target_scaler.transform(y_test)

# Train Random Forest models for latitude and longitude
models = []
target_names = ['Latitude', 'Longitude', 'Magnitude']

# Train only latitude and longitude models with Random Forest
for i in range(2):  # Only for latitude and longitude (indices 0 and 1)
    print(f"\nTraining Random Forest for {target_names[i]}...")
    
    # Create and train the model
    rf_model = RandomForestRegressor(
        n_estimators=100,        # Number of trees
        max_depth=15,            # Maximum depth of trees
        min_samples_split=5,     # Minimum samples required to split
        min_samples_leaf=2,      # Minimum samples required at leaf node
        max_features='sqrt',     # Max features to consider for best split
        n_jobs=-1,              # Use all available processors
        random_state=42          # For reproducibility
    )
    
    # Fit the model to the training data
    rf_model.fit(X_train_scaled, y_train_scaled[:, i])
    models.append(rf_model)
    
    # Make predictions on validation set to check performance
    val_pred = rf_model.predict(X_val_scaled)
    val_mse = mean_squared_error(y_val_scaled[:, i], val_pred)
    val_rmse = np.sqrt(val_mse)
    val_mae = mean_absolute_error(y_val_scaled[:, i], val_pred)
    val_r2 = r2_score(y_val_scaled[:, i], val_pred)
    
    print(f"Validation RMSE: {val_rmse:.4f}")
    print(f"Validation MAE: {val_mae:.4f}")
    print(f"Validation R²: {val_r2:.4f}")

#=== LSTM-Wavelet Model for Magnitude Prediction ===
print("\nTraining LSTM-Wavelet Model for Magnitude...")

# Create a time series representation for the magnitude data
mag_data = df[['time', 'magnitudo']].copy()
mag_data.set_index('time', inplace=True)
# Resample to monthly data for wavelet analysis (adjust frequency if needed)
monthly_mag = mag_data['magnitudo'].resample('M').mean().dropna()

# Normalize magnitude data
mag_scaler = MinMaxScaler()
mag_data_scaled = mag_scaler.fit_transform(monthly_mag.values.reshape(-1, 1)).flatten()

# Apply Discrete Wavelet Transform (DWT)
wavename = 'db4'  # Daubechies wavelet
coeffs = pywt.wavedec(mag_data_scaled, wavename, level=3)
cA3, cD3, cD2, cD1 = coeffs  # Approximation and details

# Reconstruct smoothed signal
smoothed_data = pywt.waverec([cA3] + [None] * 3, wavename)[:len(mag_data_scaled)]

# Prepare sequences for LSTM
sequence_length = 30
X_lstm, y_lstm = [], []
for i in range(len(smoothed_data) - sequence_length):
    X_lstm.append(smoothed_data[i:i+sequence_length])
    y_lstm.append(smoothed_data[i+sequence_length])
X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

# Reshape input for LSTM
X_lstm = X_lstm.reshape(X_lstm.shape[0], X_lstm.shape[1], 1)

# Split into train and test sets
split = int(0.8 * len(X_lstm))
X_lstm_train, X_lstm_test = X_lstm[:split], X_lstm[split:]
y_lstm_train, y_lstm_test = y_lstm[:split], y_lstm[split:]

# Build LSTM model for magnitude prediction
lstm_model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(sequence_length, 1)),
    Dropout(0.3),
    LSTM(100, return_sequences=False),
    Dropout(0.3),
    Dense(1)
])

# Compile and train the LSTM model
lstm_model.compile(optimizer='adam', loss='mse')
lstm_history = lstm_model.fit(
    X_lstm_train, y_lstm_train, 
    epochs=100, 
    batch_size=32, 
    validation_data=(X_lstm_test, y_lstm_test),
    verbose=1
)

# Evaluate LSTM-Wavelet model
y_lstm_pred = lstm_model.predict(X_lstm_test)
y_lstm_test_actual = y_lstm_test.reshape(-1, 1)

# Scale back to original magnitude values
y_lstm_test_denorm = mag_scaler.inverse_transform(y_lstm_test_actual)
y_lstm_pred_denorm = mag_scaler.inverse_transform(y_lstm_pred)

# Calculate magnitude prediction metrics
lstm_mae = mean_absolute_error(y_lstm_test_denorm, y_lstm_pred_denorm)
lstm_rmse = np.sqrt(mean_squared_error(y_lstm_test_denorm, y_lstm_pred_denorm))
lstm_mape = np.mean(np.abs((y_lstm_test_denorm - y_lstm_pred_denorm) / np.maximum(0.0001, y_lstm_test_denorm))) * 100
lstm_accuracy = 100 - lstm_mape  # Accuracy as (100 - MAPE)

print("\n=== LSTM-Wavelet Magnitude Model Performance ===")
print(f"Mean Absolute Error (MAE): {lstm_mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {lstm_rmse:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {lstm_mape:.2f}%")
print(f"Forecast Accuracy: {lstm_accuracy:.2f}%")

# Append LSTM model as the magnitude predictor
models.append(lstm_model)  # Note: This is the LSTM model, not a RandomForest

# Evaluate Random Forest models on test set
print("\n=== Random Forest Models Performance (Latitude & Longitude) ===")

for i, (model, target_name) in enumerate(zip(models[:2], target_names[:2])):  # Only evaluate the RF models
    # Make predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    
    # Convert predictions to original scale for a single target
    y_train_true_single = y_train_scaled[:, i]
    y_test_true_single = y_test_scaled[:, i]
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train_true_single, y_train_pred)
    test_mse = mean_squared_error(y_test_true_single, y_test_pred)
    train_rmse = np.sqrt(train_mse)
    test_rmse = np.sqrt(test_mse)
    train_mae = mean_absolute_error(y_train_true_single, y_train_pred)
    test_mae = mean_absolute_error(y_test_true_single, y_test_pred)
    train_r2 = r2_score(y_train_true_single, y_train_pred)
    test_r2 = r2_score(y_test_true_single, y_test_pred)
    
    # Overfitting ratio
    mse_overfitting_ratio = test_mse / train_mse if train_mse > 0 else float('inf')
    
    print(f"\n{target_name} Performance:")
    print(f"  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}")
    print(f"  Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
    print(f"  Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}")
    print(f"  MSE Overfitting Ratio (Test/Train): {mse_overfitting_ratio:.2f}")

# Visualize feature importances for Random Forest models
plt.figure(figsize=(14, 8))

for i, (model, target_name) in enumerate(zip(models[:2], target_names[:2])):  # Only for RF models
    plt.subplot(2, 1, i+1)
    
    # Get feature importances
    importances = model.feature_importances_
    indices = np.argsort(importances)
    
    plt.barh(range(len(indices)), importances[indices], color='skyblue')
    plt.yticks(range(len(indices)), [features[i] for i in indices])
    plt.title(f'Feature Importance for {target_name}')
    plt.xlabel('Relative Importance')
    plt.tight_layout()

plt.savefig('random_forest_feature_importance.png')
plt.close()

# Visualize LSTM training history
plt.figure(figsize=(12, 6))
plt.plot(lstm_history.history['loss'], label='Training Loss')
plt.plot(lstm_history.history['val_loss'], label='Validation Loss')
plt.title('LSTM-Wavelet Model Loss for Magnitude Prediction')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.grid(True)
plt.savefig('lstm_wavelet_training_history.png')
plt.close()

# Visualize LSTM magnitude predictions
plt.figure(figsize=(12, 6))
# Plot only a subset for clarity
n_points = min(100, len(y_lstm_test_denorm))
plt.plot(y_lstm_test_denorm[-n_points:], label='Actual Magnitude', marker='o', linestyle='-', alpha=0.7)
plt.plot(y_lstm_pred_denorm[-n_points:], label='Predicted Magnitude', marker='x', linestyle='-', alpha=0.7)
plt.title('Magnitude Prediction: Actual vs. Predicted')
plt.xlabel('Sample Index')
plt.ylabel('Magnitude')
plt.legend()
plt.grid(True)
plt.savefig('lstm_wavelet_magnitude_predictions.png')
plt.close()

# Combined prediction function
def predict_earthquake(lat_lon_features, historical_mag_sequence, models, feature_scaler, mag_scaler):
    """
    Make predictions using both the Random Forest models (lat, lon) and LSTM-Wavelet model (magnitude)
    
    Parameters:
    lat_lon_features: Numpy array of shape (n_samples, n_features) for lat/lon prediction
    historical_mag_sequence: Sequence of magnitude values of length 'sequence_length' for LSTM
    models: List of models [lat_rf_model, lon_rf_model, mag_lstm_model]
    feature_scaler: Fitted scaler for RF input features
    mag_scaler: Fitted scaler for magnitude
    
    Returns:
    prediction: Dictionary with predicted latitude, longitude, and magnitude
    """
    # Scale the input features for Random Forest
    scaled_input = feature_scaler.transform(lat_lon_features)
    
    # Get lat/lon predictions from Random Forest
    lat_pred = models[0].predict(scaled_input)
    lon_pred = models[1].predict(scaled_input)
    
    # Prepare input for LSTM (magnitude prediction)
    # Normalize the historical magnitudes
    mag_sequence_scaled = mag_scaler.transform(historical_mag_sequence.reshape(-1, 1)).flatten()
    
    # Apply same wavelet transform as in training
    wavename = 'db4'
    coeffs = pywt.wavedec(mag_sequence_scaled, wavename, level=3)
    cA3, cD3, cD2, cD1 = coeffs
    smoothed_seq = pywt.waverec([cA3] + [None] * 3, wavename)[:len(mag_sequence_scaled)]
    
    # Reshape for LSTM input [batch, timesteps, features]
    lstm_input = smoothed_seq.reshape(1, len(smoothed_seq), 1)
    
    # Predict magnitude
    mag_pred_scaled = models[2].predict(lstm_input)
    mag_pred = mag_scaler.inverse_transform(mag_pred_scaled)[0][0]
    
    return {
        'latitude': lat_pred[0],
        'longitude': lon_pred[0],
        'magnitude': mag_pred
    }

print("\nCombined earthquake prediction model trained successfully.")
print("Use the predict_earthquake() function to make new predictions.")
print("\n=== Model Performance Summary ===")
print("Latitude & Longitude: Random Forest model")
print("Magnitude: LSTM model with Wavelet transform")
print("\nThe combined approach addresses overfitting in magnitude prediction")
print("while maintaining high accuracy for location prediction.")

C:\Users\jidub\AppData\Local\Temp\ipykernel_22840\223186086.py:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\jidub\AppData\Local\Temp\ipykernel_22840\223186086.py:81: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)



Training Random Forest for Latitude...
Validation RMSE: 0.0257
Validation MAE: 0.0137
Validation R²: 0.9992

Training Random Forest for Longitude...
Validation RMSE: 0.0194
Validation MAE: 0.0100
Validation R²: 0.9994

Training LSTM-Wavelet Model for Magnitude...


C:\Users\jidub\AppData\Local\Temp\ipykernel_22840\223186086.py:164: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_mag = mag_data['magnitudo'].resample('M').mean().dropna()
C:\Users\jidub\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 126ms/step - loss: 0.1557 - val_loss: 0.0051
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.0249 - val_loss: 0.0144
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0175 - val_loss: 0.0050
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.0137 - val_loss: 0.0111
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0145 - val_loss: 0.0060
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0108 - val_loss: 0.0069
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0114 - val_loss: 0.0077
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0111 - val_loss: 0.0053
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0097 - val_loss: 0.0048
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0099 - val_loss: 0.0052
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0094 - val_loss: 0.0046
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/ste

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
import pywt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load dataset
# Note: You'll need to replace this path with the location of your earthquake dataset
df = pd.read_csv(r"C:\Users\jidub\OneDrive\Documents\Timeseries\project\Eartquakes-1990-2023.csv\Eartquakes-1990-2023.csv")

# Preprocess timestamp
df['time'] = pd.to_datetime(df['time'], unit='ms')
df = df.sort_values('time')  # Ensure data is chronologically ordered
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour

# Handle missing values
df.fillna(method='ffill', inplace=True)

# === Feature Engineering ===
# 1. Add cyclical time features
df['month_sin'] = np.sin(df['month'] * (2 * np.pi / 12))
df['month_cos'] = np.cos(df['month'] * (2 * np.pi / 12))
df['hour_sin'] = np.sin(df['hour'] * (2 * np.pi / 24))
df['hour_cos'] = np.cos(df['hour'] * (2 * np.pi / 24))
df['day_sin'] = np.sin(df['day'] * (2 * np.pi / 31))
df['day_cos'] = np.cos(df['day'] * (2 * np.pi / 31))

# 2. Rolling statistics
for window in [7, 30, 90]:
    df[f'rolling_mag_mean_{window}d'] = df['magnitudo'].rolling(window=window).mean()
    df[f'rolling_depth_mean_{window}d'] = df['depth'].rolling(window=window).mean()
    df[f'rolling_quake_count_{window}d'] = df['magnitudo'].rolling(window=window).count()

# 3. Handle outliers
def cap_outliers(series, lower_quantile=0.01, upper_quantile=0.99):
    lower_bound = series.quantile(lower_quantile)
    upper_bound = series.quantile(upper_quantile)
    return series.clip(lower=lower_bound, upper=upper_bound)

df['magnitudo'] = cap_outliers(df['magnitudo'])
df['depth'] = cap_outliers(df['depth'])

# 4. Create interaction features
df['lat_lon_interaction'] = df['latitude'] * df['longitude'] / 10000  # Scaled down

# 5. Create magnitude categories
df['mag_low'] = (df['magnitudo'] < 2.0).astype(int)
df['mag_medium'] = ((df['magnitudo'] >= 2.0) & (df['magnitudo'] < 4.0)).astype(int)
df['mag_high'] = (df['magnitudo'] >= 4.0).astype(int)

# 6. Add geographic region clustering
def assign_region(lat, lon):
    # Simplified region assignment based on latitude/longitude
    if lat > 30 and lon > 120:  # Pacific Ring of Fire - Asia side
        return 1
    elif lat > 30 and lon < -100:  # Pacific Ring of Fire - Americas side
        return 2
    elif -30 < lat < 30:  # Equatorial regions
        return 3
    else:  # Other regions
        return 4

df['region'] = df.apply(lambda x: assign_region(x['latitude'], x['longitude']), axis=1)
df = pd.get_dummies(df, columns=['region'], prefix='region')

# 7. Add squared and cubed terms for longitude model (to capture non-linear relationships)
df['longitude_squared'] = df['longitude'] ** 2
df['depth_squared'] = df['depth'] ** 2

# Fill any remaining NaN values
df.fillna(method='bfill', inplace=True)

# Define features
features = [
    'latitude', 'longitude', 'depth',  
    'month_sin', 'month_cos', 'day_sin', 'day_cos', 'hour_sin', 'hour_cos',
    'rolling_mag_mean_30d', 'rolling_depth_mean_30d', 'rolling_quake_count_30d',
    'rolling_mag_mean_7d', 'rolling_mag_mean_90d',
    'lat_lon_interaction', 'mag_low', 'mag_medium', 'mag_high',
    'region_1', 'region_2', 'region_3', 'region_4',
    'longitude_squared', 'depth_squared'  # Added polynomial features
]

targets = ['latitude', 'longitude', 'magnitudo']

# Extract features and targets
X = df[features].values
y = df[targets].values

# Use time-based split for temporal data
test_size = int(0.1 * len(X))
val_size = int(0.1 * len(X))

X_train = X[:-test_size-val_size]
X_val = X[-test_size-val_size:-test_size]
X_test = X[-test_size:]

y_train = y[:-test_size-val_size]
y_val = y[-test_size-val_size:-test_size]
y_test = y[-test_size:]

# Scale the features
feature_scaler = RobustScaler()
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Scale targets for evaluation consistency
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train)
y_val_scaled = target_scaler.transform(y_val)
y_test_scaled = target_scaler.transform(y_test)

# Train Random Forest for latitude (keep the original)
models = []

# Latitude model 
print("\nTraining Random Forest for Latitude...")
lat_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    n_jobs=-1,
    random_state=42
)
lat_model.fit(X_train_scaled, y_train_scaled[:, 0])
models.append(lat_model)

# Train Gradient Boosting for longitude to reduce overfitting
print("\nTraining Gradient Boosting for Longitude...")
lon_model = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.05,  # Lower learning rate to reduce overfitting
    max_depth=6,         # Reduced depth to avoid overfitting
    min_samples_split=10,
    min_samples_leaf=5,
    subsample=0.8,       # Use 80% of samples for each tree
    max_features=0.7,    # Use 70% of features for each split
    random_state=42
)
lon_model.fit(X_train_scaled, y_train_scaled[:, 1])
models.append(lon_model)

# Validate models
target_names = ['Latitude', 'Longitude', 'Magnitude']
for i, (model, target_name) in enumerate(zip(models, target_names[:2])):
    # Validate with cross-validation
    cv_scores = cross_val_score(model, X_val_scaled, y_val_scaled[:, i], 
                               cv=5, scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores.mean())
    
    # Make predictions on validation set
    val_pred = model.predict(X_val_scaled)
    val_mse = mean_squared_error(y_val_scaled[:, i], val_pred)
    val_rmse = np.sqrt(val_mse)
    val_mae = mean_absolute_error(y_val_scaled[:, i], val_pred)
    val_r2 = r2_score(y_val_scaled[:, i], val_pred)
    
    print(f"\n{target_name} Validation Metrics:")
    print(f"  Validation RMSE: {val_rmse:.4f}")
    print(f"  Cross-Val RMSE: {cv_rmse:.4f}")
    print(f"  Validation MAE: {val_mae:.4f}")
    print(f"  Validation R²: {val_r2:.4f}")

#=== LSTM-Wavelet Model for Magnitude Prediction ===
print("\nTraining LSTM-Wavelet Model for Magnitude...")

# Create a time series representation for the magnitude data
mag_data = df[['time', 'magnitudo']].copy()
mag_data.set_index('time', inplace=True)
# Resample to monthly data for wavelet analysis
monthly_mag = mag_data['magnitudo'].resample('M').mean().dropna()

# Normalize magnitude data
mag_scaler = MinMaxScaler()
mag_data_scaled = mag_scaler.fit_transform(monthly_mag.values.reshape(-1, 1)).flatten()

# Apply Discrete Wavelet Transform (DWT)
wavename = 'db4'  # Daubechies wavelet
coeffs = pywt.wavedec(mag_data_scaled, wavename, level=3)
cA3, cD3, cD2, cD1 = coeffs  # Approximation and details

# Reconstruct smoothed signal
smoothed_data = pywt.waverec([cA3] + [None] * 3, wavename)[:len(mag_data_scaled)]

# Prepare sequences for LSTM
sequence_length = 30
X_lstm, y_lstm = [], []
for i in range(len(smoothed_data) - sequence_length):
    X_lstm.append(smoothed_data[i:i+sequence_length])
    y_lstm.append(smoothed_data[i+sequence_length])
X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

# Reshape input for LSTM
X_lstm = X_lstm.reshape(X_lstm.shape[0], X_lstm.shape[1], 1)

# Split into train and test sets
split = int(0.8 * len(X_lstm))
X_lstm_train, X_lstm_test = X_lstm[:split], X_lstm[split:]
y_lstm_train, y_lstm_test = y_lstm[:split], y_lstm[split:]

# Build LSTM model for magnitude prediction with regularization
lstm_model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(sequence_length, 1), 
         recurrent_regularizer=tf.keras.regularizers.l2(0.01)),  # Added L2 regularization
    Dropout(0.4),  # Increased dropout rate
    LSTM(100, return_sequences=False, 
         recurrent_regularizer=tf.keras.regularizers.l2(0.01)),  # Added L2 regularization
    Dropout(0.4),  # Increased dropout rate
    Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01))  # Added L2 regularization
])

# Compile and train the LSTM model
lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
lstm_history = lstm_model.fit(
    X_lstm_train, y_lstm_train, 
    epochs=100, 
    batch_size=32, 
    validation_data=(X_lstm_test, y_lstm_test),
    callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True)],  # Added early stopping
    verbose=1
)

# Evaluate LSTM-Wavelet model
y_lstm_pred = lstm_model.predict(X_lstm_test)
y_lstm_test_actual = y_lstm_test.reshape(-1, 1)

# Scale back to original magnitude values
y_lstm_test_denorm = mag_scaler.inverse_transform(y_lstm_test_actual)
y_lstm_pred_denorm = mag_scaler.inverse_transform(y_lstm_pred)

# Calculate magnitude prediction metrics
lstm_mae = mean_absolute_error(y_lstm_test_denorm, y_lstm_pred_denorm)
lstm_rmse = np.sqrt(mean_squared_error(y_lstm_test_denorm, y_lstm_pred_denorm))
lstm_mape = np.mean(np.abs((y_lstm_test_denorm - y_lstm_pred_denorm) / np.maximum(0.0001, y_lstm_test_denorm))) * 100
lstm_accuracy = 100 - lstm_mape  # Accuracy as (100 - MAPE)

print("\n=== LSTM-Wavelet Magnitude Model Performance ===")
print(f"Mean Absolute Error (MAE): {lstm_mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {lstm_rmse:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {lstm_mape:.2f}%")
print(f"Forecast Accuracy: {lstm_accuracy:.2f}%")

# Append LSTM model as the magnitude predictor
models.append(lstm_model)  # Note: This is the LSTM model, not a RandomForest

# Evaluate models on test set
print("\n=== Models Performance Evaluation ===")

for i, (model, target_name) in enumerate(zip(models[:2], target_names[:2])):
    # Make predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    
    # Convert predictions to original scale for a single target
    y_train_true_single = y_train_scaled[:, i]
    y_test_true_single = y_test_scaled[:, i]
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train_true_single, y_train_pred)
    test_mse = mean_squared_error(y_test_true_single, y_test_pred)
    train_rmse = np.sqrt(train_mse)
    test_rmse = np.sqrt(test_mse)
    train_mae = mean_absolute_error(y_train_true_single, y_train_pred)
    test_mae = mean_absolute_error(y_test_true_single, y_test_pred)
    train_r2 = r2_score(y_train_true_single, y_train_pred)
    test_r2 = r2_score(y_test_true_single, y_test_pred)
    
    # Overfitting ratio
    mse_overfitting_ratio = test_mse / train_mse if train_mse > 0 else float('inf')
    
    print(f"\n{target_name} Performance:")
    print(f"  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}")
    print(f"  Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
    print(f"  Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}")
    print(f"  MSE Overfitting Ratio (Test/Train): {mse_overfitting_ratio:.2f}")

# Visualize feature importances for models
plt.figure(figsize=(14, 8))

# For Random Forest (latitude)
plt.subplot(2, 1, 1)
importances = models[0].feature_importances_
indices = np.argsort(importances)
plt.barh(range(len(indices)), importances[indices], color='skyblue')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.title('Feature Importance for Latitude (Random Forest)')
plt.xlabel('Relative Importance')

# For Gradient Boosting (longitude)
plt.subplot(2, 1, 2)
if hasattr(models[1], 'feature_importances_'):  # GradientBoostingRegressor has this
    importances = models[1].feature_importances_
    indices = np.argsort(importances)
    plt.barh(range(len(indices)), importances[indices], color='lightgreen')
    plt.yticks(range(len(indices)), [features[i] for i in indices])
    plt.title('Feature Importance for Longitude (Gradient Boosting)')
    plt.xlabel('Relative Importance')

plt.tight_layout()
plt.savefig('model_feature_importance.png')
plt.close()

# Visualize LSTM training history
plt.figure(figsize=(12, 6))
plt.plot(lstm_history.history['loss'], label='Training Loss')
plt.plot(lstm_history.history['val_loss'], label='Validation Loss')
plt.title('LSTM-Wavelet Model Loss for Magnitude Prediction')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.grid(True)
plt.savefig('lstm_wavelet_training_history.png')
plt.close()

# Visualize predictions
plt.figure(figsize=(15, 15))

# Visualize latitude prediction
plt.subplot(3, 1, 1)
y_lat_pred = models[0].predict(X_test_scaled)
test_indices = np.random.choice(range(len(y_test_scaled)), min(100, len(y_test_scaled)), replace=False)
plt.plot(y_test_scaled[test_indices, 0], label='Actual', marker='o', linestyle='', alpha=0.7)
plt.plot(y_lat_pred[test_indices], label='Predicted', marker='x', linestyle='', alpha=0.7)
plt.title('Latitude - Actual vs Predicted')
plt.xlabel('Sample Index')
plt.ylabel('Latitude (scaled)')
plt.legend()
plt.grid(True)

# Visualize longitude prediction
plt.subplot(3, 1, 2)
y_lon_pred = models[1].predict(X_test_scaled)
plt.plot(y_test_scaled[test_indices, 1], label='Actual', marker='o', linestyle='', alpha=0.7)
plt.plot(y_lon_pred[test_indices], label='Predicted', marker='x', linestyle='', alpha=0.7)
plt.title('Longitude - Actual vs Predicted')
plt.xlabel('Sample Index')
plt.ylabel('Longitude (scaled)')
plt.legend()
plt.grid(True)

# For magnitude, we use a subset of the LSTM test data
plt.subplot(3, 1, 3)
n_points = min(100, len(y_lstm_test_denorm))
plt.plot(y_lstm_test_denorm[-n_points:], label='Actual', marker='o', linestyle='-', alpha=0.7)
plt.plot(y_lstm_pred_denorm[-n_points:], label='Predicted', marker='x', linestyle='-', alpha=0.7)
plt.title('Magnitude - Actual vs Predicted')
plt.xlabel('Sample Index')
plt.ylabel('Magnitude')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('model_predictions.png')
plt.close()

# Combined prediction function
def predict_earthquake(lat_lon_features, historical_mag_sequence, models, feature_scaler, mag_scaler):
    """
    Make predictions using Random Forest (latitude), Gradient Boosting (longitude) 
    and LSTM-Wavelet (magnitude) models
    
    Parameters:
    lat_lon_features: Numpy array of shape (n_samples, n_features) for lat/lon prediction
    historical_mag_sequence: Sequence of magnitude values of length 'sequence_length' for LSTM
    models: List of models [lat_rf_model, lon_gbr_model, mag_lstm_model]
    feature_scaler: Fitted scaler for input features
    mag_scaler: Fitted scaler for magnitude
    
    Returns:
    prediction: Dictionary with predicted latitude, longitude, and magnitude
    """
    # Scale the input features
    scaled_input = feature_scaler.transform(lat_lon_features)
    
    # Get lat/lon predictions
    lat_pred = models[0].predict(scaled_input)
    lon_pred = models[1].predict(scaled_input)
    
    # Prepare input for LSTM (magnitude prediction)
    # Normalize the historical magnitudes
    mag_sequence_scaled = mag_scaler.transform(historical_mag_sequence.reshape(-1, 1)).flatten()
    
    # Apply same wavelet transform as in training
    wavename = 'db4'
    coeffs = pywt.wavedec(mag_sequence_scaled, wavename, level=3)
    cA3, cD3, cD2, cD1 = coeffs
    smoothed_seq = pywt.waverec([cA3] + [None] * 3, wavename)[:len(mag_sequence_scaled)]
    
    # Reshape for LSTM input [batch, timesteps, features]
    lstm_input = smoothed_seq.reshape(1, len(smoothed_seq), 1)
    
    # Predict magnitude
    mag_pred_scaled = models[2].predict(lstm_input)
    mag_pred = mag_scaler.inverse_transform(mag_pred_scaled)[0][0]
    
    return {
        'latitude': lat_pred[0],
        'longitude': lon_pred[0],
        'magnitude': mag_pred
    }

print("\nCombined earthquake prediction model trained successfully.")
print("Use the predict_earthquake() function to make new predictions.")
print("\n=== Model Performance Summary ===")
print("Latitude: Random Forest model")
print("Longitude: Gradient Boosting model (reduced overfitting)")
print("Magnitude: LSTM model with Wavelet transform")
print("\nThe combined approach addresses overfitting in both longitude and magnitude prediction")
print("while maintaining high accuracy for overall earthquake prediction.")

C:\Users\jidub\AppData\Local\Temp\ipykernel_22840\1647483206.py:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\jidub\AppData\Local\Temp\ipykernel_22840\1647483206.py:85: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)



Training Random Forest for Latitude...

Training Gradient Boosting for Longitude...

Latitude Validation Metrics:
  Validation RMSE: 0.0241
  Cross-Val RMSE: 0.0287
  Validation MAE: 0.0125
  Validation R²: 0.9993

Longitude Validation Metrics:
  Validation RMSE: 0.0050
  Cross-Val RMSE: 0.0049
  Validation MAE: 0.0034
  Validation R²: 1.0000

Training LSTM-Wavelet Model for Magnitude...


C:\Users\jidub\AppData\Local\Temp\ipykernel_22840\1647483206.py:188: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_mag = mag_data['magnitudo'].resample('M').mean().dropna()
C:\Users\jidub\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 68ms/step - loss: 2.0416 - val_loss: 1.5397
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.4538 - val_loss: 1.1545
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.0860 - val_loss: 0.8607
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.8122 - val_loss: 0.6446
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.6091 - val_loss: 0.4825
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.4572 - val_loss: 0.3642
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.3482 - val_loss: 0.2770
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.2660 - val_loss: 0.2123
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.2069 - val_loss: 0.1634
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.1609 - val_loss: 0.1275
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.1258 - val_loss: 0.1003
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step